# Part 0: Setup Data Owner 2
First of all we need to establish a connection between the points. Duet we will establish an P2P connecton. DS will be able to perform remore execurion on the DOs's data on the DS's machine.

### 0.2 Lauch the server
Now we do the same procedure as before. Keep in mind that you can simplify this part by using only one argument *loopback=True* in duet at the launch

In [1]:
import syft as sy
#duet = sy.duet(loopback=True)
duet = sy.duet(network_url="http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000")

#"http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000"
#"http://ec2-3-138-111-189.us-east-2.compute.amazonaws.com:5000"
#"http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000"

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: c42fcad4f6c305ad1921bf250a72048e

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("c42fcad4f6c305ad1921bf250a72048e")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!
♫♫♫ > Duet Partner's Client ID: 8e1852e377bd999f66c3843a47574814

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



#### Now return to the DS notebook to get and insert Server ID (Jump 0.4). Then come back to insert DS's ID (Jump 0.5). When the connection is established, get to the Part 1 in DO1 notebook (Jump 0.6)

# Part 1: Data Prepreparation DO2

### 1.3 Check the data ( DO2)
Next we import the neccessary libraries. For this example we will use the wine dataset(https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html).

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import torch as th

We split the data into the features (X) and the target labels (y). Than we convert it into a pandas dataframe in order to displey the data

In [3]:
wine = datasets.load_wine()
_, X2, _, y2 = train_test_split(wine.data, wine.target, test_size=0.6, random_state=1)
print(pd.DataFrame(X2, columns=wine.feature_names).mean())
pd.DataFrame(X2, columns=wine.feature_names)

alcohol                          13.015421
malic_acid                        2.290935
ash                               2.349533
alcalinity_of_ash                19.062617
magnesium                       100.915888
total_phenols                     2.310561
flavanoids                        2.085327
nonflavanoid_phenols              0.360561
proanthocyanins                   1.686355
color_intensity                   5.012056
hue                               0.950991
od280/od315_of_diluted_wines      2.650467
proline                         764.738318
dtype: float64


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,13.69,3.26,2.54,20.0,107.0,1.83,0.56,0.50,0.80,5.88,0.96,1.82,680.0
1,12.42,1.61,2.19,22.5,108.0,2.00,2.09,0.34,1.61,2.06,1.06,2.96,345.0
2,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845.0
3,12.21,1.19,1.75,16.8,151.0,1.85,1.28,0.14,2.50,2.85,1.28,3.07,718.0
4,13.77,1.90,2.68,17.1,115.0,3.00,2.79,0.39,1.68,6.30,1.13,2.93,1375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,13.68,1.83,2.36,17.2,104.0,2.42,2.69,0.42,1.97,3.84,1.23,2.87,990.0
103,13.16,3.57,2.15,21.0,102.0,1.50,0.55,0.43,1.30,4.00,0.60,1.68,830.0
104,13.07,1.50,2.10,15.5,98.0,2.40,2.64,0.28,1.37,3.70,1.18,2.69,1020.0
105,13.05,3.86,2.32,22.5,85.0,1.65,1.59,0.61,1.62,4.80,0.84,2.01,515.0


### 1.4 Woking with Pysyft ( DO2)
The important part is that PySyft is designed to work with tensors. Fortunately, there are several libraries in python, such as Pytorch, which are very well designed to work with them. So, what we need to do is to convert the data into tensors to work with it.


In [4]:
X2 = th.FloatTensor(X2)
y2 = th.FloatTensor(y2)
print("Length of dataset:", len(X2))

Length of dataset: 107


Even though we try to preserve privacy as much as possible, the DS, obviously, must know what kind of data he is supposed to work with. We will provide some description to the data and the tags, which the DS will use to refer to the data.

In [5]:
X2 = X2.tag("Wine-data")
y2 = y2.tag("Wine-labels")

X2 = X2.describe(
    "Wine quality dataset #2 with 13 features (Alcohol, Malic acid, Ash, Alcalinity of ash, Magnesium, Total phenols, Flavanoids, Nonflavanoid phenols, Proanthocyanins, Color intensity, Hue, OD280/OD315 of diluted wines, Proline)")
y2 = y2.describe("Labels for wine: class_0, class_1, class_2")

Next step is to upload the data to the Duet server and create pointers to the data. So every time the DS will want to use somehow the data, we will recieve a notification which we can accept or reject, meaning that the DS can not use the data without our permission. 

In [6]:
data_pointer2 = X2.send(duet, pointable=True)#searchable=True
target_pointer2 = y2.send(duet, pointable=True)#searchable=True

In [7]:
# Data owner 2 can verify that the object stored in the tensor
duet.store

[<syft.proxy.torch.TensorPointer object at 0x000001C3145DE970>, <syft.proxy.torch.TensorPointer object at 0x000001C3145DEB80>]

Similarly to the dataframe format in pandas, we display an information about the tensors.

In [8]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 602738e8807e4eb08ea630794820f471>,[Wine-data],Wine quality dataset #2 with 13 features (Alco...,<class 'torch.Tensor'>
1,<UID: 885d601cf5644ebd9a1efcf46144bd9e>,[Wine-labels],"Labels for wine: class_0, class_1, class_2",<class 'torch.Tensor'>


[2022-04-22T18:18:01.897601+0200][CRITICAL][logger]][8456] You do not have permission to .get() Object with ID: <UID: 77a22d7279074758b0daca088b45956d>Please submit a request.
[2022-04-22T18:18:01.899060+0200][CRITICAL][logger]][8456] You do not have permission to .get() Object with ID: <UID: 77a22d7279074758b0daca088b45956d>Please submit a request.


#### Now go and continue working with the DS notebook (Jump 1.2)

# Part 2: Working With The Data (DO2)
If we take a look at a DUET LIVE STATUS we will see that there is a request waiting for our approval. Let us check its content.

In [9]:
duet.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[Wine-data, mean, __getitem__]",I would be very gratefull if you could possibl...,<UID: 78dd3307835a4e7a9e0a4945e0db3dad>,<UID: 77a22d7279074758b0daca088b45956d>,


In [10]:
duet.requests[0].request_description

'I would be very gratefull if you could possibly provide me the average alcohol for my research OwO'

The request looks reasonable and we accept it

In [11]:
duet.requests[0].accept()

#### Return to the DS notebook and proceed (Jump 2.4)

# Part 3: Creating and training the models (DO 2)

The DO can add requests to be accepted or denied by adding them to request handlers. On the contrrary, if we don't specify anyhing, than it means we are willing to accept everything

In [12]:
duet.requests.add_handler(action="accept")

#### Get to the DS notebook (Jump 3.3)